In [3]:
import torch
from torch import nn

print(torch.__version__)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

2.4.0+cu121


'cuda'

# GP Run for Image Classification
Your problem needs to fulfill the following criteria.
1. It is an image classification problem.
2. You supply marked training images and marked validation images.

Within those, the run is flexible and adapts itself to your problem.

In [10]:
# The problem-specific variables to be set once per problem
IMAGE_WIDTH = 28 # <-- number of width-pixels
IMAGE_HEIGHT = 28 # <-- number of height-pixels
COLOUR_CHANNEL_COUNT = 1 # <-- RGB images would have 3
CLASSIFICATION_CATEGORIES_COUNT = 10 # <-- the amount of possible categories of which each image shall be marked with one

## The Individuals' Class
- The Hyperparamters should be passed to the constructor in a way that is both convenient for GP and for PyTorch.
- I think I want to define a class for one `nn.Sequential` 2d-block
    - All possible instances should be concatenable with all possible instances
- Then, an individual is built from the concatenation of many such blocks, plus data preparation and final f.c. layer

In [11]:
''' 
define a λ nn.Module that creates an nn layer from a given function
this is handy for nn.Sequential usage '''
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
    def forward(self, x):
        return self.func(x)
    
'''
create a function to reshape the (28x28) image input data '''
def preprocess(x):
    return x.view(-1, 1, 28, 28)

In [14]:
class Sequential_block_2d(nn.Module):
    def __init__(self, in_channels: int,
                 out_channels: int, # the number of output neurons after the full block
                 conv_kernel_size: int = 3,
                 conv_stride: int = 1,
                 conv_padding: int = 1,
                 pool_kernel_size: int = 2,
                 pool_stride: int = 2,
                 pool_padding: int = 0):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels=out_channels,
                      kernel_size=conv_kernel_size,
                      stride=conv_stride,
                      padding=conv_padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=pool_kernel_size,
                         stride=pool_stride,
                         padding=pool_padding)
        )
    def forward(self, x):
        return self.block(x)

# generate some testing blocks (TODO: write actual unit tests)
testBlock1 = Sequential_block_2d(in_channels=1,out_channels=5).to(device)
testBlock2 = Sequential_block_2d(in_channels=5,out_channels=3).to(device)
torch.manual_seed(42)
testX = torch.randn(1,IMAGE_WIDTH,IMAGE_HEIGHT).to(device)
testBlock1, testBlock2, testX, testBlock1(testX)
print(f"testX.shape = {testX.shape}\ntestX[:,:3]: {testX[:,:3]}")
print(f"testBlock1(testX).shape = {testBlock1(testX).shape}\ntestBlock1(testX)[:1,:3]: {testBlock1(testX)[:1,:3]}")

testX.shape = torch.Size([1, 28, 28])
testX[:,:3]: tensor([[[ 1.9269,  1.4873,  0.9007, -2.1055,  0.6784, -1.2345, -0.0431,
          -1.6047, -0.7521,  1.6487, -0.3925, -1.4036, -0.7279, -0.5594,
          -0.7688,  0.7624,  1.6423, -0.1596, -0.4974,  0.4396, -0.7581,
           1.0783,  0.8008,  1.6806,  1.2791,  1.2964,  0.6105,  1.3347],
         [-0.2316,  0.0418, -0.2516,  0.8599, -1.3847, -0.8712, -0.2234,
           1.7174,  0.3189, -0.4245,  0.3057, -0.7746, -1.5576,  0.9956,
          -0.8798, -0.6011, -1.2742,  2.1228, -1.2347, -0.4879, -0.9138,
          -0.6581,  0.0780,  0.5258, -0.4880,  1.1914, -0.8140, -0.7360],
         [-1.4032,  0.0360, -0.0635,  0.6756, -0.0978,  1.8446, -1.1845,
           1.3835,  1.4451,  0.8564,  2.2181,  0.5232,  0.3466, -0.1973,
          -1.0546,  1.2780, -0.1722,  0.5238,  0.0566,  0.4263,  0.5750,
          -0.6417, -2.2064, -0.7508,  0.0109, -0.3387, -1.3407, -0.5854]]],
       device='cuda:0')
testBlock1(testX).shape = torch.Size([5, 14,

In [21]:
# class for image classification individuals
class NN_individual(nn.Module):
    def __init__(self, in_dimensions: int,
                 out_dimensions: int,
                 blocks_2d: list[Sequential_block_2d],
                 fin_res: int = 5): # output dimension of the final max pooling 
                                    # producing size (fin_res * fin_res)
        super().__init__()
        self.blocks_2d = blocks_2d
        # add a final max pool
        # Why? Because then torch handles the dimensions through the "adaptive"ness
        self.last_max_pool_adaptive = nn.AdaptiveAvgPool2d((fin_res, fin_res))
        self.flatten = nn.Flatten(start_dim=0, end_dim=-1) # default start_dim = 1
        self.lin = nn.Linear(in_features = fin_res * fin_res * blocks_2d[-1].out_channels,
                      out_features = out_dimensions)
    def forward(self, x):
        for i in range(len(self.blocks_2d)):
            x = self.blocks_2d[i](x)
        x = self.last_max_pool_adaptive(x)
        #print(f"last_max_pool_adaptive output shape is {x.shape}")
        x = self.flatten(x)
        #print(f"flatten output shape is {x.shape}")
        x = self.lin(x)
        #print(f"lin output shape is {x.shape}")
        return x
    
testIndividual = NN_individual(in_dimensions=COLOUR_CHANNEL_COUNT,
    out_dimensions=CLASSIFICATION_CATEGORIES_COUNT,
    blocks_2d=[testBlock1, testBlock2]).to(device) # the individual's genes
testIndividual(testX)

tensor([ 0.0145, -0.2088,  0.1622,  0.0619,  0.1288, -0.2957, -0.1214,  0.0652,
        -0.2806, -0.0815], device='cuda:0', grad_fn=<ViewBackward0>)